In [1]:
# %pylab

import numpy
import matplotlib
from matplotlib import pyplot
np = numpy
plt = pyplot
from pylab import *
from numpy import *
from scipy.optimize import curve_fit
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
matplotlib.rcParams['figure.figsize'] = [10.5,7]
matplotlib.rcParams.update({'font.size': 25})

In [4]:
%run my_functions

# Load data file names
I load the file names of data. You have to write the data directory in `file_path` then the program sorts the file names into several categories, according to the their SimuType, and their nature.

### SimuType
- `Langevin` is the first initialisation of the cloud with the Langevin program.
- `SimuTypeQ` is the first part with velocity-Verlet. Introduces laser cooling and in a second time a ramp on the RF voltage.
- `SimuType4` is the second part. Introduces the Giant Molecule with the proper kinetic energy. Laser cooling and trapping parameters remains constant. Giant Molecule evacuated at the end of `SimuType4`.
- `SimuType2` is the third part. Laser cooling and trapping parameters remains constant.

### Nature of files
- `xva` contain a snapshot of the cloud : positions, velocities and accelerations of each ion at a given time, at the end of each SimuType, under the format `.bin`. For `SimuType4`, files with `xva` and `.dat` extension contain information about the Giant Molecule.
- `Temp` contain temperature of cloud as a function of time, under the form of a `.dat`.
- `trj` contain trajectory as a function of time : `trj` alone is for one or many Ca+ ions, `trjGMol` is the trajectory of the GMol.

In [87]:
# for i,j in enumerate(os.walk(file_path)):
#     print(i,j)
file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20230322'+'/DC10_RF10'+'/Try00'
# print(os.listdir(file_path))

file_name_Lan = sort([x for x in os.listdir(file_path) if 'Langevin' in x or '3D' in x])
print(file_name_Lan)
file_name_SimuTypeQ = sort([x for x in os.listdir(file_path) if 'SimuTypeQ' in x])
print(file_name_SimuTypeQ)
file_name_SimuType4_xva = sort([x for x in os.listdir(file_path) if 'SimuType4' in x and 'xva' in x])
print(file_name_SimuType4_xva)
file_name_SimuType4_trjGMol = sort([x for x in os.listdir(file_path) if 'SimuType4' in x and 'trj' in x and 'GMol' in x])
print(file_name_SimuType4_trjGMol)
file_name_SimuType2_xva = sort([x for x in os.listdir(file_path) if 'SimuType2' in x and 'xva' in x])
print(file_name_SimuType2_xva)

file_name_SimuType4_Temp = sort([x for x in os.listdir(file_path) if 'SimuType4' in x and 'Temp' in x])
print(file_name_SimuType4_Temp)
file_name_SimuType2_Temp = sort([x for x in os.listdir(file_path) if 'SimuType2' in x and 'Temp' in x])
print(file_name_SimuType2_Temp)

['Langevin_cooling.info'
 'Temp_3D_Harmo_N01024_T***uK_F0.15D-19Kg_sRFG.bin'
 'xva_3D_Harmo_N01024_T***uK_F0.15D-19Kg_sRFG.bin']
['Temp_SimuTypeQ_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG.dat'
 'xva_SimuTypeQ_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG.bin'
 'xva_SimuTypeQ_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG.info']
['xva_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_0.000E+00_50eV.bin'
 'xva_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_0.000E+00_50eV.dat'
 'xva_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_0.000E+00_50eV.info'
 'xva_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_1.000E-04_50eV.bin'
 'xva_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_1.000E-04_50eV.dat'
 'xva_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_1.000E-04_50eV.info'
 'xva_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_1.000E-05_50eV.bin'
 'xva_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1

# Reading snapshots

In [216]:
x0 = 0e-6 # [0, 1e-6, 5e-6, 10e-6, 50e-6, 100e-6]
str_x0 = f'x0_{x0:.3E}'
print(str_x0)
xva_Langevin  = load_xva_init_bin_Lan(file_path+'/'+sort(file_name_Lan)[2].strip('.bin'),1)[5]
xva_SimuTypeQ = load_xyz_init_bin_DP(file_path+'/'+sort(file_name_SimuTypeQ)[1].strip('.bin'))[0]
xva_SimuType4 = load_xyz_init_bin_DP(file_path+'/'+sort([x for x in file_name_SimuType4_xva if str_x0 in x])[0].strip('.bin'))[0]
xva_SimuType2 = load_xyz_init_bin_DP(file_path+'/'+sort([x for x in file_name_SimuType2_xva if str_x0 in x])[0].strip('.bin'))[0]

x0_0.000E+00


In [218]:
to_plot = xva_SimuType4
figure('xva'); clf()
# title(fig_title)
subplot(211,aspect=1.0)
plot(to_plot [0,:]*1e3,to_plot [1,:]*1e3,'8',color='xkcd:purplish blue')
xlabel('x[µm]')
ylabel('y[µm]')
grid()
# xlim(-1000,1000)
# ylim(-1000,1000)

# subplot(212,aspect=1.0)
subplot(212)
plot(to_plot [2,:]*1e3,to_plot [0,:]*1e3,'8',color='xkcd:purplish blue')
xlabel('z[µm]')
ylabel('x[µm]')
grid()
# xlim(-1000,1000)
# ylim(-1000,1000)

tight_layout()
savefig('xva_4.png',dpi=300)

In [345]:
max(to_plot [2,:]*1e3) - min(to_plot [2,:]*1e3)

497.9277379746295

# Reading temperature

In [77]:
Temp_Langevin  = load_Temp_init_bin_Lan(file_path+'/'+sort(file_name_Lan)[1].strip('.bin'),1)[5]
Temp_SimuTypeQ = load_T_and_PM_simu(file_path+'/'+sort(file_name_SimuTypeQ)[0].strip('.dat'))
Temp_SimuType4 = load_T_and_PM_simu(file_path+'/'+sort([x for x in file_name_SimuType4_Temp if str_x0 in x])[0].strip('.dat'))
Temp_SimuType2 = load_T_and_PM_simu(file_path+'/'+sort([x for x in file_name_SimuType2_Temp if str_x0 in x])[0].strip('.dat'))
print(sort(file_name_SimuTypeQ)[0])

[4] [4] [8] [8] [8] [55440]
N_ions 1024
j_save 990
dt_j_save_next 5e-07
eta 1.5e-20
Temp [0.001]
[5.05000000e-07 7.34632700e-05 1.42830264e-05 3.07188526e-05
 1.19546147e+02 4.23126514e+01 2.36321204e+02]
Temp_SimuTypeQ_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG.dat


In [33]:
figure('Temperature_Lan',clear='True')
# subplot(211)
semilogy(Temp_Langevin[:,0]*1e3,mean(Temp_Langevin[:,4:7],axis=1))
grid()
xlabel('t [ms]')
ylabel('T [K]')
tight_layout()

In [201]:
figure('Temperature',clear='True')
# subplot(211)
semilogy(Temp_Langevin[:,0]*1e3,mean(Temp_Langevin[:,4:7],axis=1))
semilogy(Temp_SimuTypeQ[0]*1e3,mean(Temp_SimuTypeQ[2],axis=1))
semilogy(Temp_SimuType4[0]*1e3,mean(Temp_SimuType4[2],axis=1))
semilogy(Temp_SimuType2[0]*1e3,mean(Temp_SimuType2[2],axis=1))
grid()
xlabel('t [ms]')
ylabel('T [K]')
tight_layout()

In [122]:
file_name_SimuType4_Temp

array(['Temp_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_0.000E+00_50eV.dat',
       'Temp_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_1.000E-04_50eV.dat',
       'Temp_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_1.000E-05_50eV.dat',
       'Temp_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_1.000E-06_50eV.dat',
       'Temp_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_5.000E-05_50eV.dat',
       'Temp_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_5.000E-06_50eV.dat'],
      dtype='<U86')

In [221]:
# En boucle

x0_list = [0, 1e-6, 5e-6, 10e-6, 50e-6, 100e-6]
my_dash = ['-','--',(0, (5, 10, 1, 10)),(0,(1,1)),':',(0,(1,10))]

figure('Temperature_loop',clear='True')

Temp_Langevin  = load_Temp_init_bin_Lan(file_path+'/'+sort(file_name_Lan)[1].strip('.bin'),1)[5]
Temp_SimuTypeQ = load_T_and_PM_simu(file_path+'/'+sort(file_name_SimuTypeQ)[0].strip('.dat'))
semilogy(Temp_Langevin[:,0]*1e3,mean(Temp_Langevin[:,4:7],axis=1))
semilogy(Temp_SimuTypeQ[0]*1e3,mean(Temp_SimuTypeQ[2],axis=1))

for i,j in enumerate(x0_list):
    str_x0 = f'{j:.3E}'
    print(i,str_x0)
    Temp_SimuType4 = load_T_and_PM_simu(file_path+'/'+sort([x for x in file_name_SimuType4_Temp if str_x0 in x])[0].strip('.dat'))
    Temp_SimuType2 = load_T_and_PM_simu(file_path+'/'+sort([x for x in file_name_SimuType2_Temp if str_x0 in x])[0].strip('.dat'))
    # subplot(211)

    semilogy(Temp_SimuType4[0]*1e3,mean(Temp_SimuType4[2],axis=1),color='C2',ls=my_dash[i],label=x0_list[i]*1e6)
    semilogy(Temp_SimuType2[0]*1e3,mean(Temp_SimuType2[2],axis=1),color='C3',ls=my_dash[i])
grid()
legend(title='$x_0$ [µm]',fontsize=15,ncol=2)
xlabel('t [ms]')
ylabel('T [K]')
xlim(8.35,9.2)
# ylim(2e-4,2e2)
tight_layout()
savefig('x0var_zoom.png',dpi=300)

[4] [4] [8] [8] [8] [55440]
N_ions 1024
j_save 990
dt_j_save_next 5e-07
eta 1.5e-20
Temp [0.001]
[5.05000000e-07 7.34632700e-05 1.42830264e-05 3.07188526e-05
 1.19546147e+02 4.23126514e+01 2.36321204e+02]
0 0.000E+00
1 1.000E-06
2 5.000E-06
3 1.000E-05
4 5.000E-05
5 1.000E-04


# Reading GMol trajectory

In [111]:
def load_trj_GMol(str_load):
    fid = open(str_load+'.bin', 'rb')
    
    junk = fromfile(fid, int32,1)        # Read record start tag
    j_save_temp  = fromfile(fid, int32,1);
    junk = fromfile(fid, int32,1)        # Read record stop tag

    junk = fromfile(fid, int32,1)        # Read record start tag
    steps  = fromfile(fid, int32,1);
    junk = fromfile(fid, int32,1)        # Read record stop tag
    
    junk = fromfile(fid, int32,1)        # Read record start tag
    print(junk)
    jmax  = fromfile(fid, int32,1);
    junk = fromfile(fid, int32,1)        # Read record stop tag
    print(junk)
    
    junk = fromfile(fid, int32,1)        # Read record start 
    print(junk)
    t_act_gmol  = fromfile(fid, float64, jmax[0]+2);
    junk = fromfile(fid, int32,1)        # Read record stop tag
    print(junk)
    
    junk = fromfile(fid, int32,1)        # Read record start tag
    print(junk)
    trj_gmol  = fromfile(fid, float64, (jmax[0]+2)*6);
    junk = fromfile(fid, int32,1)        # Read record stop tag
    print(junk)
    
    junk = fromfile(fid, int32,1)        # Read record start tag
    print(junk)
    Ep_gmol  = fromfile(fid, float64, jmax[0]+2);
    junk = fromfile(fid, int32,1)        # Read record stop tag
    print(junk)

    fid.close

    return j_save_temp, steps, jmax, t_act_gmol, trj_gmol, Ep_gmol

In [141]:
for i,j in enumerate(x0_list):
#     file_name_SimuType4_trjGMol
    str_x0 = f'{j:.3E}'
    tmp = [x for x in file_name_SimuType4_trjGMol if str_x0 in x][0].strip('.bin')
    print(tmp)

trj_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_0.000E+00_50eV_GMol
trj_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_1.000E-06_50eV_GMol
trj_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_5.000E-06_50eV_GMol
trj_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_1.000E-05_50eV_GMol
trj_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_5.000E-05_50eV_GMol
trj_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_1.000E-04_50eV_GMol


In [144]:
time_gmol = []
trj_gmol = []
steps_gmol = []

for i,j in enumerate(x0_list):
#     file_name_SimuType4_trjGMol
    str_x0 = f'{j:.3E}'
    tmp = [x for x in file_name_SimuType4_trjGMol if str_x0 in x][0].strip('.bin')
    str_load = f'{file_path}/{tmp}'

    print(str_load)
    # steps, t_act_gmol, Ep_gmol, trj_gmol = load_trj_GMol(str_load)
    j_save_temp, steps, jmax, t_act_gmol, trj_gmol_tmp, Ep_gmol = load_trj_GMol(str_load)
    trj_gmol.append( reshape(trj_gmol_tmp,(6,(jmax[0]+2)),order='F') )
    time_gmol.append( t_act_gmol )
    steps_gmol.append( steps[0] )

/home/adrien/RemoteFS/Rivendel/Simulations/20230322/DC10_RF10/Try00/trj_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_0.000E+00_50eV_GMol
[4]
[4]
[800816]
[800816]
[4804896]
[4804896]
[800816]
[800816]
/home/adrien/RemoteFS/Rivendel/Simulations/20230322/DC10_RF10/Try00/trj_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_1.000E-06_50eV_GMol
[4]
[4]
[800816]
[800816]
[4804896]
[4804896]
[800816]
[800816]
/home/adrien/RemoteFS/Rivendel/Simulations/20230322/DC10_RF10/Try00/trj_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_5.000E-06_50eV_GMol
[4]
[4]
[800816]
[800816]
[4804896]
[4804896]
[800816]
[800816]
/home/adrien/RemoteFS/Rivendel/Simulations/20230322/DC10_RF10/Try00/trj_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0RFG_Q01_x0_1.000E-05_50eV_GMol
[4]
[4]
[800816]
[800816]
[4804896]
[4804896]
[800816]
[800816]
/home/adrien/RemoteFS/Rivendel/Simulations/20230322/DC10_RF10/Try00/trj_SimuType4_01_N01024_Vrf0061_Udc0.2631D+01V_D1.0_S1.0

In [206]:
figname = 'GMol_trajectory_xz'
fig = plt.figure(figname,clear='True')
# fig.set_size_inches(11.69,8.27)
ax = fig.add_subplot(111)
# subplot(211)
# to_plot = xva_SimuTypeQ
# ax.plot(to_plot[2,:]*1e3,to_plot[0,:]*1e6,'8',color='xkcd:purplish blue')

for k in range(len(file_name_SimuType4_trjGMol)):
    ax.plot(trj_gmol[k][2][:steps_gmol[k]]*1e6,trj_gmol[k][0][:steps_gmol[k]]*1e9-x0_list[k]*1e9, # -x0_list[k]*1e9
         '+',label=x0_list[k]*1e6)
    
ax.grid()
ax.set_xlabel('z [µm]')
ax.set_ylabel('x-$x_0$ [nm]')
# xlim(-255,-181)
ylim(-72,72)
ax.legend(title='$x_0$ [µm]',fontsize=15,ncol=2)
tight_layout()
# plt.savefig(figname+'_zoom.png',dpi=300)

In [176]:
figname = 'GMol_trajectory_yz'
fig = plt.figure(figname,clear='True')
fig.set_size_inches(11.69,8.27)
ax = fig.add_subplot(111)
for k in range(len(file_name_SimuType4_trjGMol)):
    ax.plot(trj_gmol[k][2][:steps_gmol[k]]*1e6,trj_gmol[k][1][:steps_gmol[k]]*1e9, # -x0_list[k]*1e9
         '+',label=x0_list[k]*1e6)
ax.grid()
ax.set_xlabel('z [µm]')
ax.set_ylabel('y [nm]')
# xlim(-255,-181)
# ylim(-72,72)
ax.legend(title='$x_0$ [µm]',fontsize=15,ncol=2)
tight_layout()
plt.savefig(figname+'.png',dpi=300)

In [177]:
figname = 'GMol_trajectory_xy'
fig = plt.figure(figname,clear='True')
fig.set_size_inches(11.69,8.27)
ax = fig.add_subplot(111)
for k in range(len(file_name_SimuType4_trjGMol)):
    ax.plot(trj_gmol[k][0][:steps_gmol[k]]*1e9,trj_gmol[k][1][:steps_gmol[k]]*1e9, # -x0_list[k]*1e9
         '+',label=x0_list[k]*1e6)
ax.grid()
ax.set_xlabel('x [nm]')
ax.set_ylabel('y [nm]')
# xlim(-255,-181)
# ylim(-72,72)
ax.legend(title='$x_0$ [µm]',fontsize=15,ncol=2)
tight_layout()
plt.savefig(figname+'.png',dpi=300)

In [186]:
figname = 'GMol_velocity_z'
fig = plt.figure(figname,clear='True')
fig.set_size_inches(11.69,8.27)
ax = fig.add_subplot(111)
# subplot(211)
for k in range(len(file_name_SimuType4_trjGMol)):
    ax.plot(trj_gmol[k][2][:steps_gmol[k]]*1e6,trj_gmol[k][5][:steps_gmol[k]],
         '',label=x0_list[k]*1e6)
ax.grid()
ax.set_xlabel('z [µm]')
ax.set_ylabel('$v_z$ [m/s]')
ax.legend(title='$x_0$ [µm]',fontsize=15,ncol=2)
ax.set_xlim(-500,500)
ax.set_ylim(98.16,98.22)
tight_layout()
plt.savefig(figname+'zoom_.png',dpi=300)

In [ ]:
sqrt(2*(50*C_e))

In [411]:
m_GM = 1e6*1.66054e-27
C_e = 1.602e-19 

E_i = 0.5*m_GM*(98.208)**2 / C_e
E_f = 0.5*m_GM*(98.193)**2 / C_e
deltaE = E_f - E_i
print(98.208-98.193)
print(deltaE*1e3)

E_i = 0.5*m_GM*(98.1347)**2 / C_e
E_f = 0.5*m_GM*(98.03)**2 / C_e
deltaE = E_f - E_i
print(98.1347-98.03)
print(deltaE*1e3)

E_i = 0.5*m_GM*(98.04)**2 / C_e
E_f = 0.5*m_GM*(97.81)**2 / C_e
deltaE = E_f - E_i
print(98.04-97.81)
print(deltaE*1e3)


E_i = 0.5*m_GM*(97.864)**2 / C_e
E_f = 0.5*m_GM*(97.350)**2 / C_e
deltaE = E_f - E_i
print(97.864-97.350)
print(deltaE*1e3)

Q_list = [1,5,10,20]
Delta_E_list = [-15.268338789319102,-106.4447813645586,-233.45772337704318,-520.0330117286001]

0.015000000000000568
-15.268338789319102
0.10469999999999402
-106.4447813645586
0.23000000000000398
-233.45772337704318
0.51400000000001
-520.0330117286001


In [412]:
def my_lin_fit(x,a,b):
    return a*x+b    

In [413]:
popt,pcov = curve_fit(my_lin_fit,
                      Q_list,
                      Delta_E_list)

figname = 'delta_E_vs_Q'
figure(figname,clear=True)
plot(Q_list,Delta_E_list,
     marker='P',ls='')
plot(linspace(0,20,50),my_lin_fit(linspace(0,20,50),*popt),label='lin. fit')
grid()
xlabel('Q')
ylabel('$\Delta E$ [eV]')
legend()
tight_layout()
savefig(figname+'.png',dpi=300)